In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

import glob
import numpy as np
import pickle
import pandas as pd

from settings import *

In [ ]:
SIGNIFICANCE_LEVEL = 5
results_dict = {}
exp_params = [(m, TS_max_dim, TS_max_len) for m in NUM_SAMPLES for TS_max_dim in DIM_STATE_SPACE for TS_max_len in LEN_SEQUENCE]
for kernel_name in KERNELS:
    for dataset_name in DATASETS:
        for m, TS_max_dim, TS_max_len in exp_params:
            result_filepath = os.path.join(RESULTS_DIR, kernel_name, '{}_M{}_L{}_D{}.pkl'.format(dataset_name, m, TS_max_len, TS_max_dim))
            if os.path.exists(result_filepath):
                with open(result_filepath, 'rb') as f:
                    result_dict = pickle.load(f)
                for hyp in ['H0', 'H1']:
                    if len(result_dict[hyp]) < NUM_REPETITIONS:
                        print('Warning: Result file {} only has {} repetitions for {}'.format(result_filepath, len(result_dict[hyp]), hyp))
                    key = (m, TS_max_len, TS_max_dim, hyp)
                    if key not in results_dict:
                        results_dict[key] = {}
                    if kernel_name not in results_dict[key]:
                        results_dict[key][kernel_name] = []
                    results = []
                    for i, rep in enumerate(result_dict[hyp]):
                        stat, hist = rep[0], rep[1]
                        perc = np.percentile(hist, 100 - SIGNIFICANCE_LEVEL)
                        results.append(int(stat > perc))
                    results_dict[key][kernel_name].append(np.mean(results))
            else:
                print('Warning: Missing result file {}...'.format(result_filepath))
                for hyp in ['H0', 'H1']:
                    key = (m, TS_max_len, TS_max_dim, hyp)
                    if key not in results_dict:
                        results_dict[key] = {}
                    if kernel_name not in results_dict[key]:
                        results_dict[key][kernel_name] = []
                    results_dict[key][kernel_name].append(float('nan'))

In [ ]:
from IPython.display import display
for m, TS_max_dim, TS_max_len in exp_params:
    for hyp in ['H0', 'H1']:
        print('Samples: {}, Length: {}, Dimension: {} - {}'.format(m, TS_max_len, TS_max_dim, hyp))
        key = (m, TS_max_len, TS_max_dim, hyp)
        df = pd.DataFrame.from_dict(results_dict[key]).rename(index={i:k for i, k in enumerate(DATASETS)})
        display(df)